# Aquila Optimizer Implementation for the Traveling Salesman Problem

Authors:  
Reuben Vandezande  
Sam Ferguson

In [7]:
# using Pkg
# Pkg.add("SpecialFunctions")
using SpecialFunctions
using Statistics

function calculate_g1()
    # rand motion, Eq. 16
    return 2 * rand() - 1  # Eq. 16
end

function calculate_g2(iteration::Integer, max_iterations::Integer)
    # Flight slope of aquila, Eq. 17
    return 2 * (1 - iteration / max_iterations)
end

function calculate_spiral_search(
    num_dimensions::Integer,
    r1::Float64,
    miu::Float64,
    w::Float64
)
    # Spiral search Eq.(9, 10)
    dim_list = [i for i in range(1, num_dimensions + 1)]
    r = r1 .+ miu .* dim_list
    theta1 = 3 * pi / 2
    phi = -w .* dim_list .+ phi0
    x = r .* sin.(phi)  
    y = r .* cos.(phi)  

    return x, y
end

function calculate_quality_function(
    iteration::Integer, 
    max_iterations::Integer
)
    # Quality function, Eq.(15)
    iteration ^ ((2 * rand() - 1) / (1 - max_iterations) ^ 2)  
end

function get_levy_flight_step(
    beta::Float64
)
    # u and v are two rand variables which follow normal distribution
    # sigma_u : standard deviation of u
    sigma = gamma(1.0 + beta) * sin(pi * beta / 2) / (gamma((1 + beta) / 2.) * beta * (2. ^ ((beta - 1) / 2)))

    # levy step, Eq. 6
    step = rand() * sigma / (abs(rand()) ^ (1 / beta))
    
    return step
end

function get_global_best(
    population::Array{Float64, 2},
    fitness::Array{Float64, 1}
)
    min_index = argmin(fitness)
    return population[min_index], fitness[min_index]
end

function expanded_exploration(
    global_best::Float64,
    iteration::Integer,
    max_iterations::Integer,
    x_mean::Array{Float64, 1}
)
    return global_best * (1 - iteration / max_iterations) + rand() * (x_mean - global_best)
end


function rand_individual(
    population::Array{Float64, 2},
    skip_index::Integer
)
    # Filter out the exclude_value
    filtered_indices = filter(x -> x != skip_index, 1:length(population))

    # Select a rand choice from the filtered array
    index = rand(filtered_indices)

    return population[index]
end


function narrowed_exploration(
    current_index::Integer,
    population::Array{Float64, 2},
    global_best::Float64,
    beta::Float64,
    r1::Float64,
    miu::Float64,
    w::Float64
)
    levy_step = get_levy_flight_step(beta)
    
    # Define spiral search
    x, y = calculate_spiral_search(size(population, 2), r1, miu, w)

    # Use Eq. 5 to compute narrowed exploration
    return global_best * levy_step .+ rand_individual(population, current_index) .+ rand() * (y - x)  
end

function expanded_exploitation(
    alpha::Float64,
    delta::Float64,
    global_best::Array,
    x_mean::Array{Float64, 1},
    upper::Array{Float64, 1},
    lower::Array{Float64, 1}
)
    # Compute expanded exploitation from Eq. 13
    return alpha .* (global_best .- x_mean) - rand() .* (rand() .* (upper .- lower) .+ lower) .* delta
end

function narrowed_exploitation(
    iteration::Integer,
    max_iterations::Integer,
    individual::Array{Float64, 1},
    global_best::Array{Float64, 1},
    beta::Float64
)
    levy_step = get_levy_flight_step(beta)

    # Quality function
    qf = calculate_quality_function(iteration, max_iterations)

    # Flight slope and rand movement parameters
    g1 = calculate_g1()
    g2 = calculate_g2(iteration, max_iterations)

    # Eq. 14
    return qf .* global_best .- (g2 .* individual .* rand()) .- g2 .* levy_step .+ rand() .* g1  
end

function evolve!(
    positions,
    fitness,
    upper::Array{Float64, 1},
    lower::Array{Float64, 1},
    iteration::Integer,
    max_iterations:: Integer,
    alpha::Float64 = 0.1,
    delta::Float64 = 0.1,
    r1::Float64 = 10,
    miu::Float64 = 0.00565,
    w::Float64 = 0.005,
    beta::Float64 = 1.5
)
    # Calculate global population values
    best_position, best_fitness = get_global_best(positions, fitness)
    
    # Generate a new population. Assume this happens in place?
    for idx in range(1, length(positions))
        # Update mean each time as individuals update positions
        x_mean = mean(positions, dims=1)

        # Find a new position for this individual
        if iteration <= (2 / 3) * max_iterations
            if rand() < 0.5
                pos_new = expanded_exploration(global_best, iteration, max_iterations, x_mean)
            else
                pos_new = narrowed_exploration(idx, positions, global_best, beta, r1, miu, w)
            end
        else
            if rand() < 0.5
                pos_new = expanded_exploitation(alpha, delta, global_best, x_mean, upper, lower)
            else
                pos_new = narrowed_exploitation(iteration, max_iterations, individual, global_best, beta)
            end
        end
        
        # Evaluate the position
        fitness_new = evaluate(pos_new)
        
        # If better, replace
        if fitness_new < fitness[idx]
            positions[idx] = pos_new
            fitness[idx] = fitness_new
            
            if fitness_new < global_best
                best_fitness = fitness_new
                best_position = pos_new                
            end
        end
    end
end

evolve! (generic function with 7 methods)